In [1]:
import torch
import gc
import wandb

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [15]:
PROJECT_ID = 'E'
MODE = 'fine-tuning'

BASE_DIR = f"/home/naoya/kaggle/czii/project/{PROJECT_ID}/{MODE}"

In [16]:
%cd {BASE_DIR}

/home/naoya/kaggle/czii/project/E/fine-tuning


/home/naoya/kaggle/czii/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [29]:
data_dir = '/home/naoya/kaggle/czii/input/numpy-dataset'

exp_nos = ['002']
# select_folds = [0, 2, 3, 4, 5, 6]
select_folds = [1]
# select_folds = [4]

model = 'Unet'
batch_size_train = 1
batch_size_valid = 16
num_workers = 16
epoch = 50 * 2
lr = 0.001

num_samples = int(16)
spatial_size = int(96)
patch_size = int(96)

# alpha = 0.7
# beta = 0.3


pre_training = False
weight_path = None
if MODE == 'fine-tuning':
    pre_training = True
    weight_path = '/home/naoya/kaggle/czii/project/E/pre-training/output/train/001/001-1/single/best_model.pth'
    data_dir = '/home/naoya/kaggle/czii/input/numpy-dataset/wbp'
    epoch = 50
    lr *= 0.1

In [30]:
for i, exp_no in enumerate(exp_nos):
    project = f'CZII-{PROJECT_ID}-{MODE}'
    output_dir = f'{BASE_DIR}/output'

    for valid_fold in select_folds:
        exp_name = f'{exp_no}-{valid_fold}'
        split = f'fold_{valid_fold}'

        !./run_with_retries.sh "python -m run.train\
            project=$project\
            pre_training=$pre_training\
            weight_path=$weight_path\
            exp_no=$exp_no\
            exp_name=$exp_name\
            dir.data_dir=$data_dir\
            dir.output_dir=$output_dir\
            split=$split\
            model=$model\
            batch_size_train=$batch_size_train\
            batch_size_valid=$batch_size_valid\
            num_samples=$num_samples\
            spatial_size=$spatial_size\
            patch_size=$patch_size\
            num_workers=$num_workers\
            epoch=$epoch\
            optimizer.lr=$lr"

        wandb.finish()

        torch.cuda.empty_cache()
        gc.collect()

/home/naoya/kaggle/czii/.venv/lib/python3.12/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.22 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
[2024-12-13 01:11:05,384][train.py][INFO] - Set Up DataModule
Loading dataset: 100%|█████████████████████████| 98/98 [00:00<00:00, 294.99it/s]
/home/naoya/kaggle/czii/project/E/fine-tuning/src/models/common.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the 